In [1]:
import pandas as pd
first3 = pd.read_csv("C:\\Users\\talfi\\.spyder-py3\\first3.csv")

In [2]:
def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(actual_class)
  roc_auc_dict = {}
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_actual_class = [0 if x in other_class else 1 for x in actual_class]
    new_pred_class = [0 if x in other_class else 1 for x in pred_class]

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
    roc_auc_dict[per_class] = roc_auc

  return roc_auc_dict

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
#logreg = LogisticRegression()
X_train, X_test, y_train ,y_test = train_test_split(
    first3.drop(columns='Company'),
    first3['Company'],
    test_size=0.25,
    random_state=42,
    stratify=first3['Company']
)

In [5]:
from tpot import TPOTClassifier
tpot = TPOTClassifier(
    generations=5,
    population_size=20,
    verbosity=3,
    scoring='accuracy',
    random_state=42,
    disable_update_check=True,
    config_dict='TPOT light'
)
tpot.fit(X_train, y_train)

# AUC score for tpot model
tpot_auc_score = roc_auc_score_multiclass(y_test, tpot.predict_proba(X_test)[:, 1])
print(f'\nAUC score:',tpot_auc_score)

# Print best pipeline steps
print('\nBest pipeline steps:', end='\n')
for idx, (name, transform) in enumerate(tpot.fitted_pipeline_.steps, start=1):
    # Print idx and transform
    print(f'{idx}. {transform}')

19 operators have been imported by TPOT.


_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only dual=False, got dual=True.

Generation 1 - Current Pareto front scores:

-1	0.8073333333333332	DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=gini, DecisionTreeClassifier__max_depth=2, DecisionTreeClassifier__min_samples_leaf=12, DecisionTreeClassifier__min_samples_split=7)

-2	0.8081333333333334	DecisionTreeClassifier(SelectPercentile(input_matrix, SelectPercentile__percentile=55), DecisionTreeClassifier__criterion=gini, DecisionTreeClassifier__max_depth=8, DecisionTreeClassifier__min_samples_leaf=17, DecisionTreeClassifier__min_samples_split=7)
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 f

In [6]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=8, min_samples_leaf=17, min_samples_split=7,
                       random_state=42)

In [7]:
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
dtc.score(X_test,y_test)

0.792